In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import math

C:\Users\ptngu\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Load data
df = []
with open("df", 'rb') as f:
    content = f.read()
    df = pickle.loads(content)
df = np.asarray(df).astype(np.float32)

m,n,o,p = df.shape

df = df.reshape((df.shape[0], n*o*p))

In [3]:
labels = []
with open("labels", 'rb') as f:
    content = f.read()
    labels = pickle.loads(content)
    labels = pd.get_dummies(labels)
labels = np.asarray(labels)

In [4]:
# Training Parameters
learning_rate = 0.001
num_steps = 700
batch_size = 64
display_step = 10
train_size = math.floor(m*0.8)

# Network Parameters
n_input = n * o * p
n_classes = 10
dropout = 0.75  # Dropout, probability to keep units

In [5]:
print(m,n,o,p)

4654 118 13 11


In [6]:
# Shuffle data
permutation = np.random.permutation(len(df))
df = df[permutation]
labels = labels[permutation]

# Split Train/Test
traindf = df[:train_size]
trainLabels = labels[:train_size]

testdf = df[train_size:]
testLabels = labels[train_size:]

In [7]:
# tf Graph input
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)  # dropout (keep probability)

In [8]:
# Create model
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def max_pool(sound, k):
    return tf.nn.max_pool(sound, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')


def conv_net(_X, _weights, _biases, _dropout):
    # Reshape input
    _X = tf.reshape(_X, shape=[-1, n, o, p])

    # Convolution Layer
    conv1 = conv2d(_X, _weights['wc1'], _biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = max_pool(conv1, k=2)
    # Apply Dropout
    #conv1 = tf.nn.dropout(conv1, _dropout)

    # Convolution Layer
    conv2 = conv2d(conv1, _weights['wc2'], _biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = max_pool(conv2, k=2)
    # Apply Dropout
    #conv2 = tf.nn.dropout(conv2, _dropout)

    # Fully connected layer
    # Reshape conv3 output to fit dense layer input
    dense1 = tf.reshape(conv2, [-1, _weights['wd1'].get_shape().as_list()[0]])
    # Relu activation
    dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, _weights['wd1']), _biases['bd1']))
    # Apply Dropout
    dense1 = tf.nn.dropout(dense1, _dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(dense1, _weights['out']), _biases['out'])
    return out

In [9]:
# Store layers weight & bias
weights = {
    # 2x2 conv, p input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([2, 2, p, 32])),
    # 2x2 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([2, 2, 32, 64])),
    # fully connected, 30*4*64 inputs, 2^10 outputs
    'wd1': tf.Variable(tf.random_normal([30*4*64, 1024])),
    # 2^10 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])+0.01),
    'bc2': tf.Variable(tf.random_normal([64])+0.01),
    'bd1': tf.Variable(tf.random_normal([1024])+0.01),
    'out': tf.Variable(tf.random_normal([n_classes])+0.01)
    }

In [10]:
# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

In [11]:
def next_batch(batch_size, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:batch_size]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [12]:
# Launch the graph
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    
    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = next_batch(batch_size, traindf, trainLabels)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.7})
        #sess.run(train_op, feed_dict={X: traindf, Y: trainLabels, keep_prob: 1.})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.7})
            #loss, acc = sess.run([cost, accuracy], feed_dict={X: traindf, Y: trainLabels, keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    saver.save(sess, "./mfcc_model")

    # Calculate accuracy
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={X: testdf, Y: testLabels, keep_prob: 1.}))

Step 1, Minibatch Loss= 8413034.0000, Training Accuracy= 0.062
Step 10, Minibatch Loss= 4974771.5000, Training Accuracy= 0.328
Step 20, Minibatch Loss= 5123604.0000, Training Accuracy= 0.266
Step 30, Minibatch Loss= 2797080.5000, Training Accuracy= 0.391
Step 40, Minibatch Loss= 2769429.0000, Training Accuracy= 0.359
Step 50, Minibatch Loss= 2193551.2500, Training Accuracy= 0.391
Step 60, Minibatch Loss= 2525865.5000, Training Accuracy= 0.328
Step 70, Minibatch Loss= 1644925.5000, Training Accuracy= 0.469
Step 80, Minibatch Loss= 1796220.3750, Training Accuracy= 0.406
Step 90, Minibatch Loss= 1771050.0000, Training Accuracy= 0.438
Step 100, Minibatch Loss= 1700742.0000, Training Accuracy= 0.359
Step 110, Minibatch Loss= 1703810.8750, Training Accuracy= 0.375
Step 120, Minibatch Loss= 1683934.8750, Training Accuracy= 0.375
Step 130, Minibatch Loss= 1264360.5000, Training Accuracy= 0.438
Step 140, Minibatch Loss= 1327556.2500, Training Accuracy= 0.359
Step 150, Minibatch Loss= 1130590.00

## Testing on test data

In [13]:
# Load test data
test = []
with open("test", 'rb') as f:
    content = f.read()
    test = pickle.loads(content)
test = np.asarray(df).astype(np.float32)

In [14]:
print(test)

[[-6.90631638e+01 -8.94353561e+01 -1.46558640e+02 ... -2.27098598e+01
  -2.49863491e+01 -3.37332573e+01]
 [-1.00561394e+02 -1.07341621e+02 -1.26368607e+02 ...  2.38584924e+00
  -7.65956020e+00 -7.83618069e+00]
 [-6.08147316e+01  1.16712656e+01  3.85840225e+01 ...  3.45299301e+01
   1.72764149e+01  7.81008339e+00]
 ...
 [-9.94997711e+01 -1.28598160e+02 -1.51047226e+02 ... -1.35269022e+01
  -7.14845419e+00 -4.67074299e+00]
 [-1.89628906e+02 -2.00777924e+02 -2.23441757e+02 ... -6.96538973e+00
  -3.96023655e+00  1.88281909e-01]
 [-1.16509964e+02 -1.70851242e+02 -2.30140564e+02 ... -1.15974646e+01
  -1.07387991e+01 -1.54411640e+01]]


In [15]:
# Predict on test data
X = tf.placeholder(tf.float32, [None, n_input])
with tf.Session() as sess:
    saver.restore(sess, "./mfcc_model")
    y_pred = sess.run(prediction, feed_dict = {X: test})

INFO:tensorflow:Restoring parameters from ./mfcc_model


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,16874]
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[?,16874], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder', defined at:
  File "C:\Users\ptngu\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\ptngu\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\ptngu\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\ptngu\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\ptngu\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Users\ptngu\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 127, in start
    self.asyncio_loop.run_forev